In [ ]:
!pip install -q transformers datasets accelerate torchvision
!pip install --upgrade accelerate
!pip install --upgrade transformers
!pip install -q sentencepiece
!pip install -q jiwer
!pip install -q evaluate

In [ ]:
import os
from PIL import Image
import numpy as np
from datasets import Dataset
import pandas as pd
import torch
from transformers import TrOCRProcessor, default_data_collator, VisionEncoderDecoderModel
import evaluate
import numpy as np
import pandas as pd
import glob as glob
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

 
 
from PIL import Image
from zipfile import ZipFile
from tqdm.notebook import tqdm
from dataclasses import dataclass
dir_path = '/kaggle/input/groundtruthtext'
test_path = dir_path + "/" + 'test/gt_text'

In [ ]:
from transformers import VisionEncoderDecoderModel
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-large-handwritten", use_fast=True)

In [ ]:

# model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")
# model_path = '/kaggle/input/trocr_base_400/pytorch/default/1/checkpoint-400'
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-large-handwritten")
# processor = TrOCRProcessor.from_pretrained(
model.to("cuda")
print(model)

In [ ]:
# folder_path = test_path + "/" + "MIT_10"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = pd.DataFrame(columns=["folder", "file", "text"])
save_cnt = 0
filecnt = 1
foldercnt = 1
for folder in os.listdir(test_path):
    folder_path = test_path + "/" + folder
    print(foldercnt, ". Running folder: ", folder)
    save_cnt += 1
    for file in os.listdir(folder_path):
        filename = file.split(".")[0]
        image_path = folder_path + "/" + file
        # print("Running File ", filename)
        try: 
            image = Image.open(image_path).convert("RGB")
            pixel_values = processor(images=image, return_tensors="pt").pixel_values.to(device)#.half()
            
            # Generate output
            generated_ids = model.generate(pixel_values)
            
            # Decode generated token ids to string
            generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
            df.loc[len(df)] = [folder, filename, generated_text]
        except:
            print(folder, "/" , filename, " is skipped")
            df.loc[len(df)] = [folder, filename, "#*#"]
        # print("Predicted Text:", generated_text)
    
    if save_cnt > 100:
        df.to_csv(f"df_{filecnt}.csv")
        filecnt += 1
        save_cnt = 0
df.to_csv(f"df_final.csv")